In [2]:
library(tidyverse)

# Pluck or chuck a single element from a vector or environment

`pluck()` and `chuck()` implement a generalised form of `[[` that allow you to index deeply and flexibly into data structures. `pluck()` consistently returns NULL when an element does not exist,`chuck()` always throws an error in that case.

```r
pluck(.x, ..., .default = NULL)

chuck(.x, ...)

pluck(.x, ...) <- value
```

**Arguments**  
`.x`, `x`	
A vector or environment

`...`	
A list of accessors for indexing into the object. Can be an integer position, a string name, or an accessor function (except for the assignment variants which only support names and positions). If the object being indexed is an S4 object, accessing it by name will return the corresponding slot.

These dots support tidy dots features. In particular, if your accessors are stored in a list, you can splice that in with !!!.

`.default`	
Value to use if target is empty or absent.

`value`	
A value to replace in .x at the pluck location.

# Details


* You can pluck or chuck with standard accessors like integer positions and string names, and also accepts arbitrary accessor functions, i.e. functions that take an object and return some internal piece.

* This is often more readable than a mix of operators and accessors because it reads linearly and is free of syntactic cruft. Compare: `accessor(x[[1]])$foo` to `pluck(x, 1, accessor, "foo")`.

* These accessors never partial-match. This is unlike `$` which will select the disp object if you write `mtcars$di`.

# Examples

In [3]:
player1 <- list('VN Champions', list(31, name = 'VN Pikachu'))

player2 <- list('King Allool', list(35, name = 'xXx-Hadi-xXx'))

players <- list(player1, player2)

players

[[1]]
[[1]][[1]]
[1] "VN Champions"

[[1]][[2]]
[[1]][[2]][[1]]
[1] 31

[[1]][[2]]$name
[1] "VN Pikachu"



[[2]]
[[2]][[1]]
[1] "King Allool"

[[2]][[2]]
[[2]][[2]][[1]]
[1] 35

[[2]][[2]]$name
[1] "xXx-Hadi-xXx"

**`pluck()`** provides a way of retrieving objects from such data
structures using a combination of numeric positions, vector or
list names, and accessor functions.

In [6]:
# Numeric positions index into the list by position, just like `[[`:

players %>% pluck(1)

#equivalent to
players[[1]]

[[1]]
[1] "VN Champions"

[[2]]
[[2]][[1]]
[1] 31

[[2]]$name
[1] "VN Pikachu"

[[1]]
[1] "VN Champions"

[[2]]
[[2]][[1]]
[1] 31

[[2]]$name
[1] "VN Pikachu"

In [8]:
players %>% pluck(1, 2)

#equivalent to:
players[[1]][[2]]

[[1]]
[1] 31

$name
[1] "VN Pikachu"

[[1]]
[1] 31

$name
[1] "VN Pikachu"

In [10]:
# Supply names to index into named vectors:
players %>% pluck(1, 2, 'name')

# Equivalent to
players[[1]][[2]]$name

[1] "VN Pikachu"

[1] "VN Pikachu"

By default, `pluck()` consistently returns `NULL` when an element

In [12]:
players %>% pluck(10)

NULL

In [14]:
# supply a default value
players %>% pluck(10, .default = '<missing>')

[1] "<missing>"

If you prefer to consistently fail for non-existing elements, use
the opinionated variant **`chuck()`** :

In [16]:
# raise an error if does not exist
try(players %>% chuck(10))

Error : Index 1 exceeds the length of plucked object (10 > 2)


The `map()` functions use `pluck()` by default to retrieve multiple

In [24]:
players %>% map(list(2, 'name'))

# equivalents:
players %>% map(function(value) pluck(value, 2, 'name'))
                
players %>% map(pluck, 2, 'name') # 2, 'name' is aditional argument passing to function `pluck`
                

players %>% map(~ pluck(., 2, 'name'))  # Using a formula

[[1]]
[1] "VN Pikachu"

[[2]]
[1] "xXx-Hadi-xXx"

[[1]]
[1] "VN Pikachu"

[[2]]
[1] "xXx-Hadi-xXx"

[[1]]
[1] "VN Pikachu"

[[2]]
[1] "xXx-Hadi-xXx"

[[1]]
[1] "VN Pikachu"

[[2]]
[1] "xXx-Hadi-xXx"

In [27]:
# You can also supply a default:

players %>% map(list(2, 'gender'), .default = '<missing>')

# Comments: I think this is not intuitive. apply function function pluck instead
players %>% map(pluck, 2, 'gender', .default = '<missing')

[[1]]
[1] "<missing>"

[[2]]
[1] "<missing>"

[[1]]
[1] "<missing"

[[2]]
[1] "<missing"

In [29]:
# Or use the strict variant:
try(players %>% map(chuck, 2, 'gender'))

Error : Can't find name `gender` in vector


In [30]:
# You can also assign a value in a pluck location with pluck<-:
pluck(players, 1, 1) <- 'VNC'

players

[[1]]
[[1]][[1]]
[1] "VNC"

[[1]][[2]]
[[1]][[2]][[1]]
[1] 31

[[1]][[2]]$name
[1] "VN Pikachu"



[[2]]
[[2]][[1]]
[1] "King Allool"

[[2]][[2]]
[[2]][[2]][[1]]
[1] 35

[[2]][[2]]$name
[1] "xXx-Hadi-xXx"

`pluck()` also supports accessor functions:

In [33]:
get_value <- function(x) x[[2]]$name

players %>% pluck(1, get_value)

[1] "VN Pikachu"

In [34]:
# If you have alist of accessors, you can splice those in with `!!!`:

players %>% pluck(!!!list(1, 2, 'name'))

[1] "VN Pikachu"